# Title

In [1]:
# Libraries used

import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import time
import numpy as np
import sys

## Scrapping

### In this section, we will scrap (https://www.gsmarena.com/makers.php3) to get all companies names, devices and number of devices.

In [ ]:
the_link = 'https://www.gsmarena.com/makers.php3'
ua = UserAgent()
def get_soup(the_link):
    source_code = requests.get(the_link, headers = {'User-agent': ua.random})
    return BeautifulSoup(source_code.text, 'html5lib')


In [ ]:
soup = get_soup(the_link)
list_of_companies = soup.find('div', {'class': 'st-text'})

company_name = []
links = []
number_of_devices = []

for anchor in list_of_companies.findAll('a'):
    company_name.append(anchor.text.strip())
    links.append('https://www.gsmarena.com/'+anchor.attrs['href'])
    number_of_devices.append(anchor.find('span').text)

We notice that companies names and the number of devices in company_name list are compined into a single string as shown below.

In [ ]:
print(company_name[:2])
print(number_of_devices[:2])

We will now clean the data

In [ ]:
for i in range(len(company_name)):
    company_name[i] = company_name[i].replace(number_of_devices[i],'')

In [ ]:
# Now data is cleaned
print(company_name[:2])
print(number_of_devices[:2])

In [ ]:
df1 = pd.DataFrame()
df1['company_name'] = company_name
df1['link'] = links
df1['number_of_devices'] = [int(i.replace('devices','').strip()) for i in number_of_devices]

In [ ]:
devices = []
companies_names = []
for a_link in df1.link.values:
    soup = get_soup(a_link)
    cn = df1[df1[link] == a_link]['company_name'].values[0]
    
    # We'll now generate a list with all the pages containing a company's device 
    all_pages_links = [a_link]
    list_of_pages = soup.find('div', {'class': 'nav-pages'})
    try:
        for i in list_of_pages.find_all('a'):
            all_pages_links.append('https://www.gsmarena.com/'+i.get('href'))
    except:
        pass
    
    # We'll get all the devices names 
    for page_link in all_pages_links:
        soup = get_soup(page_link)
        list_of_devices = soup.find('div', {'class': 'makers'})
        time.sleep(1+np.random.ranint(0,3))
        try:
            for name in list_of_devices.find_all('span'):
                time.sleep(1+np.random.ranint(0,3))
                devices.append(name.text)
                companies_names.append(cn)
        except:
            devices.append(list_of_pages.find('span').text)
    time.sleep(1+np.random.ranint(0,3))
    
devices

In [ ]:
# Now we will create a dataframe that contains the companies names and the names of the devices
CN_D = pd.DataFrame({'CN':companies_names, 'D':devices})

In [ ]:
# Now we will join the two dataframes
df_devices = pd.merge(df1, CN_D, on='company_name')
df_devices

### In the next section, we will scrap Wikipedia to get the financial information we want for each device.

In [ ]:
# wikipedia Scrapping

### In the next section, we will scrap YouTube to get the length of the video, number of like and dislikes, the channels names, and the devices the reviews were about.

In [ ]:
# YouTube Scrapping

## Regression